In [201]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm
import time

In [2]:
import rotation_matrix as rm

In [3]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [4]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [36]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/CellFlask/Flask_35_200_22/TCP.txt', delimiter=',', skiprows=1)

In [37]:
data.shape[0]

1544

In [38]:
#start_pos = [-0.32840330749723673,
# 0.6906975268273259,
# 0.25610656751907307,
# -1.2222948307378443,
# -1.1666822479428505,
# -1.218172452957232]

#rtde_c.moveL(start_pos, 0.5, 0.5)

In [39]:

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [112]:
data

array([[2.42233196e-08, 8.29891660e-05, 5.00000000e-04],
       [8.47816440e-08, 1.66002462e-04, 1.00000000e-03],
       [1.93786506e-07, 2.48991628e-04, 1.50000000e-03],
       ...,
       [1.93786506e-07, 2.48991628e-04, 1.50001000e-03],
       [8.47816440e-08, 1.66002462e-04, 1.00001000e-03],
       [2.42233196e-08, 8.30132960e-05, 5.00013000e-04]])

In [113]:
# start_pos = rtde_r.getActualTCPPose()

start_pos = [-0.34957822507297887,
 0.6821259171088855,
 0.22494216457758412,
 -1.2155551165943093,
 -1.176864822380718,
 -1.2292597140201593]

In [143]:
start_pos = rtde_r.getActualTCPPose()

In [158]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    positions.append([-data[i,1], -data[i,0], 0.0, 0.0, 0.0, data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [159]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [160]:
positions_converted

[[-0.06396156351953272,
  0.1359934769886002,
  0.5521968137710105,
  2.2324564615203166,
  -0.022196255024436326,
  -2.2083529573241165],
 [-0.06396246031133532,
  0.13599440863222512,
  0.5522798170166381,
  2.2322011682920913,
  -0.022751823621351214,
  -2.2081053824747956],
 [-0.06396335640138258,
  0.13599538846455783,
  0.5523627956313931,
  2.2319458056963826,
  -0.02330726667828529,
  -2.2078577378969735],
 [-0.06396425243256601,
  0.13599640489148132,
  0.5524457990115935,
  2.2316903739297027,
  -0.023862584214117633,
  -2.2076100237851115],
 [-0.06396514794673293,
  0.13599744525450366,
  0.5525287747110705,
  2.2314348730021405,
  -0.024417776245898,
  -2.2073622401493767],
 [-0.06396604321179064,
  0.1359985464610713,
  0.5526117777178841,
  2.2311793028409816,
  -0.02497284278958093,
  -2.2071143869180974],
 [-0.06396693776137495,
  0.13599969584190266,
  0.5526947548239808,
  2.2309236635223186,
  -0.025527783862697342,
  -2.206866464166445],
 [-0.06396783233757831,
  0.

In [204]:
# get first 5 x and z positions 
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3] for x in positions]


#positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*89)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

[8.299219137334405e-05, 8.299260179687699e-05, 8.301578659636175e-05, 8.299338726929382e-05, 8.299387527330747e-05, 8.299412961577109e-05, 8.301751832102985e-05, 8.299521847834216e-05, 8.299565608257436e-05, 8.29961089578417e-05, 8.299673871627783e-05, 8.299722866405615e-05, 8.302076524645963e-05, 8.299843534002582e-05]
[0.00738630503222762, 0.007386341559922052, 0.007388405007076196, 0.00738641146696715, 0.007386454899324365, 0.007386477535803627, 0.007388559130571656, 0.0073865744445724526, 0.007386613391349118, 0.00738665369724791, 0.0073867097457487275, 0.0073867533511009975, 0.007388848106934907, 0.007386860745262298]
Speed in m/s:  0.007386997722436363


In [205]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif positions_converted[i] == positions_converted[i-1] == positions_converted[i-2] == positions_converted[i-3] == positions_converted[i-4]:
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

770
9


In [206]:
# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

In [199]:
# print length of both lists
print(len(positions_converted1))
print(len(positions_converted2))

770
774


In [194]:
positions

[[-0.06397497340924937, 0.13601183363611494, 0.5534416241142702],
 [-0.06397586432143422, 0.1360133826427184, 0.5535245970657844],
 [-0.06397675479231726, 0.1360149801332936, 0.5536075695132503],
 [-0.06397764517996779, 0.13601661415688326, 0.5536905644408119],
 [-0.06397853487943957, 0.13601829639916815, 0.5537735360071836],
 [-0.06397942413784072, 0.13602002710002725, 0.5538565070697712],
 [-0.0639803131759377, 0.13602178200477585, 0.5539394778807306],
 [-0.06398120202030337, 0.13602358558258001, 0.5540224710455756],
 [-0.06398209017649027, 0.13602543737907943, 0.5541054408492307],
 [-0.06398297800187419, 0.13602732551950947, 0.5541884102750476],
 [-0.06398386549691748, 0.13602924995307503, 0.5542713793235545],
 [-0.06398475255065897, 0.1360312228706123, 0.5543543478680132],
 [-0.06398563927405983, 0.13603323208128512, 0.5544373160351618],
 [-0.06398652580349817, 0.13603528999041103, 0.5545203065559319],
 [-0.06398741175502541, 0.13603738400358867, 0.5546032738414581]]

In [195]:
velocity = avg_speed
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
path_pose1 = [start_pos[0], start_pos[1], start_pos[2], start_pos[3], start_pos[4], start_pos[5], velocity, acceleration, blend_1]
path = []
path.append(path_pose1)
for i in range(data.shape[0]-1):
    path.append([positions_converted[i][0], positions_converted[i][1], positions_converted[i][2], positions_converted[i][3], positions_converted[i][4], positions_converted[i][5], velocity, acceleration, blend_i])

path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
path.append(path_pose3)

In [211]:
velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(len(positions_converted1)-1):
    path.append([positions_converted1[i][0], positions_converted1[i][1], positions_converted1[i][2], positions_converted1[i][3], positions_converted1[i][4], positions_converted1[i][5], velocity, acceleration, blend_i])

path.append([positions_converted1[-1][0], positions_converted1[-1][1], positions_converted1[-1][2], positions_converted1[-1][3], positions_converted1[-1][4], positions_converted1[-1][5], velocity, acceleration, 0])
rtde_c.moveL(path)

time.sleep(count/60)

path_2 = []
for i in range(len(positions_converted2)-1):
    path_2.append([positions_converted2[i][0], positions_converted2[i][1], positions_converted2[i][2], positions_converted2[i][3], positions_converted2[i][4], positions_converted2[i][5], velocity, acceleration, blend_i])

path_2.append([positions_converted2[-1][0], positions_converted2[-1][1], positions_converted2[-1][2], positions_converted2[-1][3], positions_converted2[-1][4], positions_converted2[-1][5], velocity, acceleration, blend_3])

rtde_c.moveL(path_2)

rtde_c.stopScript()

In [208]:
# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()